<a href="https://colab.research.google.com/github/ufofon2/catalog-service/blob/main/05_rag_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 환경변수 설정


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

### Pinecone 클라이언트 초기화


In [ ]:
from pinecone import Pinecone

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

In [ ]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "wiki"

# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

### 검색하기 (Pinecone)


In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 객체를 초기화합니다.
# 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)

# 질문 리스트를 정의합니다.
question = ["벨기에(Belgium)는 어디 있나요?"]

# 질문을 임베딩하여 벡터로 변환합니다.
embedded_question = embedding.embed_documents(question)

# 임베딩된 질문 벡터를 출력합니다.
print(embedded_question)

In [ ]:
# Pinecone 인덱스에서 쿼리를 실행합니다.
query_result = index.query(
    namespace="wiki-ns1",  # wiki-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
    top_k=5,  # 상위 5개의 결과를 반환합니다.
    include_vector=False,  # 벡터를 포함하지 않습니다.
    include_metadata=True  # 메타데이터를 포함합니다.
)

# 쿼리 결과에서 각 매치의 ID를 추출하여 리스트로 만듭니다.
result_ids = [r.id for r in query_result.matches]
print(result_ids)  # 결과 ID를 출력합니다.

In [ ]:
# 쿼리 결과에서 각 매치의 점수와 메타데이터를 출력합니다.
for r in query_result.matches:
    print(r.score, r.metadata)

### 검색하기 (LangChain)


In [ ]:
from langchain_pinecone import PineconeVectorStore

# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "full_text"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="full_text")

# 질문에 대한 유사도 검색을 수행합니다.
# query는 question 리스트의 첫 번째 요소를 사용하고, 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question[0], k=5, namespace="wiki-ns1")
print(docs)

# 검색된 문서의 메타데이터를 출력합니다.
for doc in docs:
    print(doc.metadata)